In [1]:
import cv2
import numpy as np
import os 
import smtplib                                              # Импортируем библиотеку по работе с SMTP                                                 
import datetime
import time
    

# Добавляем необходимые подклассы - MIME-типы
import mimetypes # Импорт класса для обработки неизвестных MIME-типов, базирующихся на расширении файла
from email import encoders # Импортируем энкодер
from email.mime.base import MIMEBase # Общий тип
from email.mime.text import MIMEText # Текст/HTML
from email.mime.image import MIMEImage # Изображения
from email.mime.multipart import MIMEMultipart # Многокомпонентный объект



def send_email(addr_to, msg_subj, msg_text, file):
    addr_from = "gloomyrobot@yandex.ru"                         # Отправитель
    password  = "Password"                                   # Пароль
    msg = MIMEMultipart()                                        # Создаем сообщение
    msg['From']    =  "SAFETY SYSTEM<gloomyrobot@yandex.ru>"     # Адресат
    msg['To']      = addr_to                                     # Получатель
    msg['Subject'] = msg_subj                                    # Тема сообщения

    body = msg_text                                         
    msg.attach(MIMEText(body, 'plain'))                     # Добавляем в сообщение текст
    attach_file(msg, file)

    #настраиваем этот блок для  почтового провайдера yandex почты
    server = smtplib.SMTP_SSL('smtp.yandex.ru', 465)        # Создаем объект SMTP
    server.login(addr_from, password)                       # Получаем доступ
    server.send_message(msg)                                # Отправляем сообщение
    server.quit()                                           # Выходим

def attach_file(msg, filepath):                             # Функция по добавлению скриншота с камеры к сообщению
    filename = os.path.basename(filepath)                   # Получаем только имя файла
    ctype, encoding = mimetypes.guess_type(filepath)        # Определяем тип файла на основе его расширения
    maintype, subtype = ctype.split('/', 1)                 # Получаем тип и подтип
    if maintype == 'image':                               
        with open(filepath, 'rb') as fp:
            img = MIMEImage(fp.read(), _subtype=subtype)
            fp.close()
        img.add_header('Content-Disposition', 'attachment', filename='screenshot '+now.strftime("%d-%m-%Y %H:%M:%S")) 
    msg.attach(img)                                        # Присоединяем файл к сообщению


recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read('trainer/trainer.yml')
cascadePath = "Python/haarcascade_frontalface_default.xml"
faceCascade = cv2.CascadeClassifier(cascadePath);

font = cv2.FONT_HERSHEY_SIMPLEX

# ID counter
id = 0

# Names related to IDs
names = ['None','Andrey', 'Katya', 'Nikto', 'Z', 'W'] 

# Initialize and start realtime video capture
cam = cv2.VideoCapture(0)

# Define min window size to be recognized as a face
#minW = 0.1*cam.get(3)
#minH = 0.1*cam.get(4)
minW = 20
minH = 20

#print(cam.get(3))
#print(cam.get(4))

#if (os.path.isfile("Python/Unknown.jpeg")==True):
#    os.remove("Python/Unknown.jpeg")

sent=False
id_check1=0
id_check2=0
ts = False
sch=0
seg=True
cropped = np.array(10)

#time.sleep(3)
while True:
    ret, img =cam.read()
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    check=0
    
    faces = faceCascade.detectMultiScale(gray, scaleFactor = 1.2, minNeighbors = 12, minSize = (int(minW), int(minH)))
    lower_color = np.array([49, 100, 0])
    upper_color = np.array([120, 255, 255])
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, lower_color, upper_color)
    nf=0
    hum=0
    cropped = [] 
    
    if (sch>100):    
        
        for(x,y,w,h) in faces:
        
            #time.sleep(3)
        
            id, confidence = recognizer.predict(gray[y:y+h,x:x+w])
            cv2.rectangle(img, (x,y), (x+w,y+h), (60, 20, 220), 2)
            cv2.imwrite("Python/Unknown.jpeg", img)


            #id_check1=id
            #print(id)
            # Check if confidence is less them 100 ==> "0" is perfect match 
            if (confidence < 70):

                #if (id_check2!=id_check1):
                check+=1



                id = names[id]
                confidence = "  {0}%".format(round(100 - confidence))

            else:
                id = "Unknown"
                #confidence = "  {0}%".format(round(100 - confidence))
                confidence = "  {0}%".format(0)
                #now = datetime.datetime.now()
                cv2.imwrite("Python/Unknown.jpeg", img)
            if (id == "Unknown"):
                cv2.putText(img, str(id), (x+5,y-5), font, 1, (255,255,255), 2)
                #cv2.putText(img, str(confidence), (x+5,y+h-5), font, 1, (255,255,0), 1)

                cropped.append(mask[x:x+w,y+h:y+2*h])
#                for pi in range(hum):
#                    cv2.imshow('Cropped',cropped[pi])
                m=len(cropped[hum])
                n=len(cropped[hum][0])
                #print('размер фрагмента (в пикселях): ',n,'*',m,'=',n*m)
                #print('из них:')
                color=0
                notcolor=0

                for i in range(m):
                    for j in range(n):
                        #print(cropped[hum][i][j])
                        if cropped[hum][i][j]  == 255:
                            color+=1
                        else:
                            notcolor+=1

                #print('пикселей нужного цвета = ',color)
                #print('пикселей не нужного цвета = ',notcolor)
                hum+=1
                if color > 0.5*n*m:
                    #seg = True
                    #print('Жилет найден')
                    cv2.putText(img, "found", (x,y+h+30), cv2.FONT_HERSHEY_SIMPLEX, 1, (60, 20, 220), 3)
                else:
                    #seg = False
                    nf+=1;
                    cv2.putText(img, "not found", (x,y+h+30), cv2.FONT_HERSHEY_SIMPLEX, 1, (60, 20, 220), 3)	

            else:
                cv2.putText(img, str(id), (x+5,y-5), font, 1, (255,255,255), 2)
                cv2.putText(img, str(confidence), (x+5,y+h-5), font, 1, (255,255,0), 1)
                cv2.rectangle(img, (x,y), (x+w,y+h), (154,255,0), 2)
            if nf>0:
                seg=False      
                   


            if check!=len(faces) and sent==False and seg==False:

                #Отправка

                # Использование функции send_email()
                #addr_to   = "khlyupina.yuliya@gmail.com" 
                addr_to   = "amlnkv@yandex.ru" 
                #addr1_to   = "khlyupina.yuliya@gmail.com"
                file = 'Python/Unknown.jpeg'   #Отправка скрина с неизвестным человеком           
                now = datetime.datetime.now()
                send_email(addr_to, "Оповещение системы безопасности", \
                "ВНИМАНИЕ!\nВ помещении обнаружен неопознанный сотрудник либо человек без специальной формы!\nВремя обнаружения: "+ \
                now.strftime("%d-%m-%Y %H:%M:%S")+ \
                "\nВозможно, стоит проверить помещение на наличие посторонних лиц.\nКадр с камеры видеонаблюдения:",\
                file)

                #send_email(addr1_to, "Оповещение системы безопасности", \
                #"ВНИМАНИЕ!\nВ помещении обнаружен неопознанный сотрудник либо человек без специальной формы!\nВремя обнаружения: "+ \
                #now.strftime("%d-%m-%Y %H:%M:%S")+ \
                #"\nВозможно, стоит проверить помещение на наличие посторонних лиц.\nКадр с камеры видеонаблюдения:",\
                #file)


                sent=True
                seg=True

            if (check==len(faces)):
                sent==False

            #print(check, ' ',len(faces))
        
    sch+=1
    cv2.imshow('Video',img) 

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
#print(len(faces))
cam.release()
cv2.destroyAllWindows()

IndexError: index 0 is out of bounds for axis 0 with size 0